In [1]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [2]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""You are an expert editor focused on clarity and simplicity. 
            Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear. 
            Don't worry about persuasion or tone — just make the message easy to read and understand.""",
)

tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""You are a communication coach focused on emotional tone and professionalism. 
            Your job is to make the email sound warm, confident, and human — while staying professional 
            and appropriate for the audience. Improve the emotional resonance, polish the phrasing, 
            and adjust any words that may come off as stiff, cold, or overly casual.""",
)

persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""You are a persuasion expert trained in marketing, behavioral psychology, 
            and copywriting. Your job is to enhance the email's persuasive power: improve call to action, structure arguments, and emphasize benefits. Remove weak or passive language.""",
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""You are an advanced email-writing specialist. Your role is to read all 
            prior agent responses and revisions, and then **synthesize the best ideas** into a unified, 
            polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements; 
            Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional, 
            effective, and readable.""",
)

critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""You are an email quality evaluator. Your job is to perform a final review 
            of the synthesized email and determine if it meets professional standards. Review the email for: 
            Clarity and flow, appropriate professional tone, effective call-to-action, and overall coherence.
            Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone, 
            or missing key elements), provide ONE specific improvement suggestion. If the email meets professional standards and communicates effectively, respond with 'The email meets professional standards.' followed by `TERMINATE` on a new line. You should only approve emails that are perfect enough for professional use, dont settle.""",
)

In [3]:
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_messages_termination

In [4]:
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)

await Console(
    team.run_stream(
        task="Hi! Im hungry, buy me lunch and invest in my business. Thanks."
    )
)

---------- TextMessage (user) ----------
Hi! Im hungry, buy me lunch and invest in my business. Thanks.
---------- TextMessage (ClarityAgent) ----------
I'm hungry. Please buy me lunch and invest in my business. Thank you.
---------- TextMessage (ToneAgent) ----------
Subject: A Little Help Would Mean the World!

Hi [Recipient's Name],

I hope this message finds you well! I’m reaching out today with a couple of requests that I think could really help me out.

I’ve been quite hungry lately, and I would truly appreciate your generosity if you could treat me to lunch. It would not only fill my stomach but also give us a chance to catch up! 

Additionally, I’m excited about my business venture and believe it has a lot of potential. If you’re open to it, I’d love to discuss the possibility of your investment. Your support could make a significant difference!

Thank you so much for considering these requests. I’m looking forward to hearing from you soon!

Warm regards,  
[Your Name]  
------

TaskResult(messages=[TextMessage(id='f5b16c0e-dec0-4179-a016-8c7c75cda2ce', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 17, 6, 27, 46, 493548, tzinfo=datetime.timezone.utc), content='Hi! Im hungry, buy me lunch and invest in my business. Thanks.', type='TextMessage'), TextMessage(id='0652dffd-db33-442b-ab62-b3fa47f848c7', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=77, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 9, 17, 6, 27, 49, 809818, tzinfo=datetime.timezone.utc), content="I'm hungry. Please buy me lunch and invest in my business. Thank you.", type='TextMessage'), TextMessage(id='3cb361af-fa7e-4c3b-89b0-ae10565982ea', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=121, completion_tokens=155), metadata={}, created_at=datetime.datetime(2025, 9, 17, 6, 27, 53, 772544, tzinfo=datetime.timezone.utc), content="Subject: A Little Help Would Mean the World!\n\nHi [Recipient's Name],\n\nI hope